In [285]:
import requests
import pandas as pd
import datetime
from statistics import mean, median, mode
import sqlite3

In [286]:
# I THINK I WILL RE-FIX THIS SCRIPT TO USE THE PIPE-DELIMITED SDN LIST
# Here's the url - https://www.treasury.gov/ofac/downloads/sdn.pip
r = requests.get('https://www.treasury.gov/ofac/downloads/sdn.del')

In [287]:
lines = str(r.text).split("\r\n")

In [288]:
matrix = []
for line in lines:
    entry = line.split("@")
    matrix.append(entry)

In [289]:
ind = []
corp = []

In [290]:
count = 0
while count < len(matrix):
    if len(matrix[count]) > 1:
        if matrix[count][2] == '"individual"':
            ind.append(matrix[count])
        else:
            corp.append(matrix[count])
    count += 1

In [291]:
f_ind = [] # f for final list for ind
for entry in ind:
    if len(entry) > 1:
        if len(entry) == 12:
            try:
                last_name = entry[1].split(',')[0].strip('"')
                first_name = entry[1].split(',')[1].strip('"')
            except IndexError as e:
                last_name = entry[1].split(',')[0].strip('"')
                first_name = ""
            f_ind.append([
                int(entry[0]), 
                last_name,
                first_name[1:],
                entry[2].strip('"'),
                entry[3].strip('"'),
                entry[4].strip('"'),
                entry[11].strip('"')
            ])
            
        else:
            try:
                last_name = entry[1].split(',')[0].strip('"')
                first_name = entry[1].split(',')[1].strip('"')
            except IndexError as e:
                last_name = entry[1].split(',')[0].strip('"')
                first_name = ""
            ent_11 = str()
            count = 11
            while count < len(entry):
                ent_11 = ent_11 + "@" + str(entry[count]) 
                count += 1
            f_ind.append([
                int(entry[0]), 
                last_name,
                first_name[1:],
                entry[2].strip('"'),
                entry[3].strip('"'),
                entry[4].strip('"'),
                ent_11[2:].strip('"')
            ])

In [292]:
f_corp = [] # f for final list for corp
for entry in corp:
    if len(entry) > 1:
        if len(entry) == 12:
            f_corp.append([
                int(entry[0]), 
                entry[1].strip('"'),
                entry[2].strip('"'),
                entry[3].strip('"'),
                entry[11].strip('"')
            ])
            
        else:
            ent_11 = str()
            count = 11
            while count < len(entry):
                ent_11 = ent_11 + "@" + str(entry[count]) 
                count += 1
            f_corp.append([
                int(entry[0]), 
                entry[1].strip('"'),
                entry[2].strip('"'),
                entry[3].strip('"'),
                ent_11[2:].strip('"')
            ])

In [293]:
df_ind = pd.DataFrame(f_ind, columns=["#", "Last Name", "First Name", "Ind/Entity", "Global Tag", "Note", "Extra"])

In [294]:
df_corp = pd.DataFrame(f_corp, columns=["#", "Name", "Type", "Country", "Extra"])

In [295]:
conn = sqlite3.connect('SDN_List.db')
c = conn.cursor()
 
def create_table_IND():
    c.execute('''CREATE TABLE IF NOT EXISTS SDNindividual(num INTEGER, lastname VARCHAR, 
                                                          firstname VARCHAR, globaltag VARCHAR,
                                                          note TEXT, extra TEXT)
              ''')
def create_table_CORP():
    c.execute('''CREATE TABLE IF NOT EXISTS SDNcorporation(num INTEGER, name VARCHAR, 
                                                          type VARCHAR, country VARCHAR,
                                                          extra TEXT)
              ''')

In [298]:
def ind_entry(df_ind):
    count = 0
    while count < len(df_ind):
        c.execute('''INSERT OR IGNORE INTO SDNindividual(num, lastname, firstname, globaltag, note, extra) 
                     VALUES(?, ?, ?, ?, ?, ?)''',
              (df_ind['#'][count], df_ind['Last Name'][count], df_ind['First Name'][count], 
               df_ind['Global Tag'][count], df_ind['Note'][count], df_ind['Extra'][count]))
        conn.commit()
        count += 1
def corp_entry(df_corp):
    count = 0
    while count < len(df_corp):
        c.execute('''INSERT OR IGNORE INTO SDNcorporation(num, name, type, country, extra) 
                     VALUES(?, ?, ?, ?, ?)''',
              (df_corp['#'][count], df_corp['Name'][count], df_corp['Type'][count],
               df_corp['Country'][count], df_corp['Extra'][count]))
        conn.commit()
        count += 1

In [299]:
create_table_IND()
ind_entry(df_ind)

In [300]:
create_table_CORP()
corp_entry(df_corp)

In [301]:
# IF NOT SQLITE DB - YOU CAN USE CSV EXPORT - UNCOMMENT BELOW
'''
date = str(datetime.date.today())
csv_ind = 'OFAC_INDIV_MATRIX_' + date + '.csv'
csv_corp = 'OFAC_CORP_MATRIX_' + date + '.csv'
csv1 = df_ind.to_csv(csv_ind, index=False)
csv2 = df_corp.to_csv(csv_corp, index=False)
'''

"\ndate = str(datetime.date.today())\ncsv_ind = 'OFAC_INDIV_MATRIX_' + date + '.csv'\ncsv_corp = 'OFAC_CORP_MATRIX_' + date + '.csv'\n"